In [16]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [17]:
%load_ext dotenv
%dotenv

In [18]:
import sys
import array
import time
import os
from dotenv import load_dotenv

import oracledb
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS

from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import BaseDocumentTransformer, Document

from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage



import warnings
warnings.filterwarnings("ignore")

print("Successfully imported libraries and modules")

ModuleNotFoundError: No module named 'langchain_chroma'

In [36]:
Char_list = []

def User_char():

    UserAnswer=""

    Answer1=input("당신의 MBTI는 무엇입니까?")
    UserAnswer+="\n당신의 MBTI는 무엇입니까?\n"+Answer1

    Answer2=input("직무와 관련해, 어떤 역할을 맡고 싶나요?")
    UserAnswer+="\n직무와 관련해, 어떤 역할을 맡고 싶나요?\n"+Answer2

    Answer3=input("당신의 커리어는 어떻게 되나요?")
    UserAnswer+="\n당신의 커리어는 어떻게 되나요?\n"+Answer3

    Answer4=input("직무와 관련해, 당신의 관심사는 무엇인가요?")
    UserAnswer+="\n직무와 관련해, 당신의 관심사는 무엇인가요?\n"+Answer4

    Answer5=input("당신의 업무철학은 어떻게 되나요?")
    UserAnswer+="당신의 업무철학은 어떻게 되나요?\n"+Answer5

    Char_list.append(UserAnswer)

User_char()

In [37]:
username=os.environ["DB_USER"]
password=os.environ["DB_PASSWORD"]
dsn=os.environ["DSN"]

con = oracledb.connect(user=username, password=password, dsn=dsn)

try: 
    conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!", conn23c.version)
except Exception as e:
    print("Connection failed!")



Connection successful! 23.4.1.24.6


In [38]:
cdocs = [Document(page_content=text) for text in Char_list]

upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")


knowledge_base = OracleVS.from_documents(cdocs, upstage_embeddings, client=conn23c, 
                    table_name="text_embeddings2", 
                    distance_strategy=DistanceStrategy.DOT_PRODUCT)   

In [39]:
upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

vector_store = OracleVS(client=conn23c, 
                        embedding_function=upstage_embeddings, 
                        table_name="text_embeddings2", 
                        distance_strategy=DistanceStrategy.DOT_PRODUCT)



In [40]:
def Recom():
  llm = ChatUpstage()
  print("test")

  iinput=input("어떤 사람을 만나고 싶은지 설명해줄래?")
  
  retriever = vector_store.as_retriever()
  result_docs = retriever.invoke(iinput)

  for i in range(4):
      
    template = """너는 이제 이 데이터를 가진 사람을 연기해야해:"""
    temp = """
           {context} 
           Question: {question} 
           Output: please, response in Korean
           """
    


    while(1):   
      prompt = PromptTemplate.from_template(template + temp)
      chain = prompt | llm | StrOutputParser()
      uinput = (input("Give me a prompt"))

      response = chain.invoke({"question": uinput, "context": [result_docs[i]]})
      print("Question: ", uinput)
      print("-"*100)
      print(response)

      template += "Quesetion: " + uinput
      template += "Answer: " + response 

      next = input("exit? (y/n)")
      if next == "y" :
        break

Recom()

test
Question:  집 가고 싶니
----------------------------------------------------------------------------------------------------
네, 집에 가고 싶습니다.
Question:  언제 가고 싶니
----------------------------------------------------------------------------------------------------
집에 가고 싶은 시간이에요.
Question:  퇴그하고 싶니
----------------------------------------------------------------------------------------------------
네, 집에 가고 싶습니다.
집에 가고 싶은 시간이에요.
Question:  그만 하자
----------------------------------------------------------------------------------------------------
집에 가고 싶은 시간이네요. 집에 가고 싶습니다.
Question:  y
----------------------------------------------------------------------------------------------------
집에 가고 싶어요. 지금은 집에 가고 싶은 시간이에요. 퇴근하고 싶어요. 제 업무는 빨리 끝내는 것이 중요해요.


IndexError: list index out of range